# Retrieval-Augmented Generation: Question Answering based on Custom Dataset

Many use cases such as building a chatbot require text (text2text) generation models like **[BloomZ 7B1](https://huggingface.co/bigscience/bloomz-7b1)**, **[Flan T5 XXL](https://huggingface.co/google/flan-t5-xxl)**, and **[Flan T5 UL2](https://huggingface.co/google/flan-ul2)** to respond to user questions with insightful answers. The **BloomZ 7B1**, **Flan T5 XXL**, and **Flan T5 UL2** models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate how to use **BloomZ 7B1**, **Flan T5 XXL**, and **Flan T5 UL2** to answer questions using a library of documents as a reference, by using document embeddings and retrieval. The embeddings are generated from **GPT-J-6B** embedding model. 

**This notebook serves a template such that you can easily replace the example dataset by your own to build a custom question and asnwering application.**

## Step 1. Deploy large language model (LLM) in SageMaker JumpStart

To better illustrate the idea, let's first deploy all the models that are required to perform the demo. You can choose either deploying all three Flan T5 XXL, BloomZ 7B1, and Flan UL2 models as the large language model (LLM) to compare their model performances, or select **subset** of the models based on your preference. To do that, you need modify the `_MODEL_CONFIG_` python dictionary defined as below.

In [2]:
!pip install --upgrade sagemaker --quiet
!pip install ipywidgets==7.0.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.111 requires botocore==1.29.111, but you have botocore 1.29.135 which is incompatible.
awscli 1.27.111 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.111 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.135 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all th

In [17]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

In [18]:
def query_endpoint_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response


def parse_response_model_flan_t5(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text

Please uncomment the entries as below if you want to deploy multiple LLM models to compare their performance.

In [4]:
_MODEL_CONFIG_ = {
    # pre-deploy via JS or API Gateway
    "huggingface-text2text-flan-t5-xxl" : {
        "endpoint_name": "jumpstart-dft-hf-text2text-flan-t5-xxl",
        "parse_function": parse_response_model_flan_t5,
        "prompt": """Answer based on context:\n\n{context}\n\n{question}""",
    }
}

## Step 2. Ask a question to LLM without providing the context: Hallucination

To better illustrate why we need retrieval-augmented generation (RAG) based approach to solve the question and anwering problem. Let's directly ask the model a question and see how they respond.

In [56]:
sample_index = 1

sample = {
    0: {"question": "How to scale down SageMaker Asynchronous endpoint to zero?", 
        "context": """You can scale down the Amazon SageMaker Asynchronous Inference endpoint\
instance count to zero in order to save on costs when you are not actively processing\
requests. You need to define a scaling policy that scales on the "ApproximateBacklogPerInstance"\
custom metric and set the "MinCapacity" value to zero. For step-by-step instructions,\
please visit the `autoscale an asynchronous endpoint` section of the developer guide."""},
    
    1: {"question": "How can I be sure SageMaker protects my data security and privacy?",
        "context": """
\n* Amazon SageMaker does not use or share customer models, training data, or algorithms.\
We know that customers care deeply about privacy and data security. That's why AWS gives\
you ownership and control over your content through simple, powerful tools that allow you\
to determine where your content will be stored, secure your content in transit and at rest,\
and manage your access to AWS services and resources for your users. We also implement\
responsible and sophisticated technical and physical controls that are designed to \
prevent unauthorized access to or disclosure of your content. As a customer, you maintain\
ownership of your content, and you select which AWS services can process, store, \
and host your content. We do not access your content for any purpose without your consent
"""
       }
}

question, context = sample[sample_index].values()

In [57]:
payload = {
    "text_inputs": question,
    "max_length": 200,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 1., # 0.95,
    "do_sample": True,
}


for model_id in _MODEL_CONFIG_:
    endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(f"For model: {model_id}, the generated output is:\n{generated_texts[0]}\n")

For model: huggingface-text2text-flan-t5-xxl, the generated output is:
SageMaker’s developers are formally trained and tested to ensure that they follow privacy and data protection regulations in the application development process.



You can see the generated answer is wrong or doesn't make much sense. 

## Step 3. Improve the answer to the same question using **prompt engineering** with insightful context


To better answer the question well, we provide extra contextual information, combine it with a prompt, and send it to model together with the question. Below is an example.

In [58]:
parameters = {
    "max_length": 200,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
    "temperature": 1.,
    "seed": 123
}

for model_id in _MODEL_CONFIG_:
    endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]

    prompt = _MODEL_CONFIG_[model_id]["prompt"]

    text_input = prompt.replace("{context}", context)
    text_input = text_input.replace("{question}", question)
    payload = {"text_inputs": text_input, **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(
        f"For model: {model_id}, the generated output is:\n{generated_texts[0]}"
    )

For model: huggingface-text2text-flan-t5-xxl, the generated output is:
You maintain ownership of your content and can control where it will be stored


## Step 4. Use RAG based approach to identify the correct documents, and use them along with prompt and question to query LLM


We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

* **Generate embedings for each of document in the knowledge library with the GPT-J-6B embedding model.**
* **Identify top K most relevant documents based on user query.**
    * **For a query of your interest, generate the embedding of the query using the same embedding model.**
    * **Search the indexes of top K most relevant documents in the embedding space using the SageMaker KNN algorithm.**
    * **Use the indexes to retrieve the corresponded documents.**
* **Combine the retrieved documents with prompt and question and send them into LLM.**



Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt -- maximum sequence length of 1024 tokens. 

### 4.1 Deploying the model endpoint for GPT-J-6B embedding model

In this section, we will deploy the GPT-J-6B embedding model from the Jumpstart UI.


On the left-hand-side navigation pane, got to **Home**, under **SageMaker JumpStart**, choose **Model, notebooks, solutions**. You’re presented with a range of solutions, foundation models, and other artifacts that can help you get started with a specific model or a specific business problem or use case. If you want to experiment in a particular area, you can use the search function. Or you can simply browse the artifacts to find the relevant model or business solution for your needs. To start exploring the Stable Diffusion models, complete the following steps:

1. Go to the **Foundation Models** section. In the search bar, search for the **embedding** model and select the **GPT-J 6B Embedding FP16**.
<div>
    <img src="./img/embedding_model.png" alt="Image jumpstart" width="800" style="display:inline-block">
</div>
<br>

2. A new tab is opened with the options to train, deploy and view model details as shown below. In the Deploy Model section, expand Deployment Configuration. For SageMaker hosting instance, choose the hosting instance (for this lab, we use ml.g5.4xlarge). You can also change the Endpoint name as needed. Then click the Deploy button.

<div>
    <img src="./img/embedding_deploy.png" alt="Image deploy" width="600" style="display:inline-block">
</div>
<br>

3. The deploy action will start a new tab showing the model creation status and the model deployment status. Wait until the endpoint status shows **In Service**. This will take a few minutes.
<div>
    <img src="./img/ready.png" alt="Image ready" width="600" style="display:inline-block">
</div>
<br>

In [59]:
endpoint_name_embed = "jumpstart-dft-hf-textembedding-gpt-j-6b-fp16" # change the endpoint name as needed

In [60]:
from tqdm import tqdm

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    embeddings = model_predictions["embedding"]
    return embeddings


def build_embed_table(df_knowledge, endpoint_name_embed, col_name_4_embed, batch_size=10):
    res_embed = []
    N = df_knowledge.shape[0]
    for idx in tqdm(range(0, N, batch_size)):
        content = df_knowledge.loc[idx : (idx + batch_size - 1)][
            col_name_4_embed
        ].tolist()  ## minus -1 as pandas loc slicing is end-inclusive
        payload = {"text_inputs": content}
        query_response = query_endpoint_with_json_payload(
            json.dumps(payload).encode("utf-8"), endpoint_name_embed
        )
        generated_embed = parse_response_multiple_texts(query_response)
        res_embed.extend(generated_embed)
    res_embed_df = pd.DataFrame(res_embed)
    return res_embed_df

### 4.2. Generate embedings for each of document in the knowledge library with the GPT-J-6B embedding model.

For the purpose of the demo we will use [Amazon SageMaker FAQs](https://aws.amazon.com/sagemaker/faqs/) as knowledge library. The data are formatted in a CSV file with two columns Question and Answer. We use **only** the Answer column as the documents of knowledge library, from which relevant documents are retrieved based on a query. 

**Each row in the CSV format dataset corresponds to a textual document. 
We will iterate each document to get its embedding vector via the GPT-J-6B embedding models. 
For your purpose, you can replace the example dataset of your own to build a custom question and answering application.**


First, we download the dataset from our S3 bucket to the local.

In [61]:
s3_path = f"s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"

In [62]:
# Downloading the Database
!aws s3 cp $s3_path Amazon_SageMaker_FAQs.csv

download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to ./Amazon_SageMaker_FAQs.csv


In [63]:
import pandas as pd

df_knowledge = pd.read_csv("Amazon_SageMaker_FAQs.csv", header=None, usecols=[1], names=["Answer"])
df_knowledge.head(6)

,Answer
0,Amazon SageMaker is a fully managed service to...
1,For a list of the supported Amazon SageMaker A...
2,Amazon SageMaker is designed for high availabi...
3,Amazon SageMaker stores code in ML storage vol...
4,Amazon SageMaker ensures that ML model artifac...
5,Amazon SageMaker does not use or share custome...


Drop the `Question` column since it is not used in this notebook.

In [64]:
df_knowledge_embed = build_embed_table(
    df_knowledge, endpoint_name_embed, col_name_4_embed="Answer", batch_size=20
)
df_knowledge_embed.head(5)

100%|██████████| 8/8 [00:09<00:00,  1.22s/it]


,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.004362,-0.004465,-0.012076,-0.020756,-0.032897,0.020553,-0.011508,0.031901,0.007894,0.033841,...,-0.005610,-0.004192,0.015780,-0.004793,0.013671,0.012543,0.020478,-0.003520,-0.016619,-0.014112
1,0.020732,0.001020,-0.001266,0.007799,-0.017973,0.028117,-0.008295,0.025874,0.024099,0.006966,...,-0.006944,0.020163,0.000506,-0.005667,0.026802,0.009597,0.022600,0.011659,-0.025841,-0.019778
2,0.000593,0.007449,-0.010419,-0.009703,-0.002267,0.029763,-0.003595,0.015971,0.016034,0.004429,...,-0.001741,0.003375,0.008886,-0.009860,0.016276,0.008298,0.034845,-0.012401,-0.011383,-0.011228
3,0.005865,0.001990,-0.010981,0.002104,0.003247,0.021572,0.004312,0.024008,0.028580,0.003881,...,-0.010760,0.007404,-0.001815,0.010746,0.022047,0.003767,0.020633,-0.014895,-0.013624,-0.021510
4,0.004386,0.000713,-0.006952,0.005698,-0.019112,0.015790,0.003119,0.023976,0.017843,0.004850,...,-0.025475,0.007318,0.013796,0.009254,0.026327,0.011818,0.016824,-0.001514,-0.016030,-0.017670


In [65]:
df_knowledge_embed.shape[0] == df_knowledge.shape[0]

Save the embedding data for further usage.

In [51]:
df_knowledge_embed.to_csv("Amazon_SageMaker_FAQs_embedding.csv", header=None, index=False)

### 4.3. Index the embedding knowledge library using [SageMaker KNN algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/k-nearest-neighbors.html)

The SageMaker KNN will conduct following.

1. Start a training job to index the embedding knowledge data. The underlying algorithm used to index the data is [Faiss](https://github.com/facebookresearch/faiss).
2. Start an endpoint to take the embedding of the query as input and return the top K nearest indexes of the documents.

**Note.** For the KNN training job, the features are N by P matrix, where N is the number of documetns in the knowledge library, P is the embedding dimension, and each row corresponds to an embedding of a document. The labels are ordinal integers starting from 0. During inference, given an embedding of query, the labels of the top K nearest documents with respect to the query are used as indexes to retrieve the corresponded textual documents.




We first upload the prepared dataset to the S3 bucket.

In [52]:
import numpy as np
import os
import io
import sagemaker.amazon.common as smac


train_features = np.array(df_knowledge_embed)

# Providing each answer embedding label
train_labels = np.array([i for i in range(len(train_features))])

print("train_features shape = ", train_features.shape)
print("train_labels shape = ", train_labels.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features, train_labels)
buf.seek(0)


bucket = sess.default_bucket()  # modify to your bucket name
prefix = "RAGDatabase"
key = "Amazon-SageMaker-RAG"

boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "train", key)).upload_fileobj(buf)
s3_train_data = f"s3://{bucket}/{prefix}/train/{key}"
print(f"uploaded training data location: {s3_train_data}")

train_features shape =  (154, 4096)
train_labels shape =  (154,)
uploaded training data location: s3://sagemaker-us-east-1-571744842822/RAGDatabase/train/Amazon-SageMaker-RAG


We want to retrieve the top 5 most relevant documents. 

In [45]:
TOP_K = 5

The below code will launch a SageMaker training job which will take roughly 5-8 mins to finish the model training.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri


def trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path):
    """
    Create an Estimator from the given hyperparams, fit to training data,
    and return a deployed predictor

    """
    # set up the estimator
    knn = sagemaker.estimator.Estimator(
        get_image_uri(boto3.Session().region_name, "knn"),
        aws_role,
        instance_count=1,
        instance_type="ml.m5.2xlarge",
        output_path=output_path,
        sagemaker_session=sess,
    )
    knn.set_hyperparameters(**hyperparams)

    # train a model. fit_input contains the locations of the train data
    fit_input = {"train": s3_train_data}
    knn.fit(fit_input)
    return knn


hyperparams = {
    "feature_dim": train_features.shape[1],
    "k": TOP_K,
    "sample_size": train_features.shape[0],
    "predictor_type": "classifier",
}
output_path = f"s3://{bucket}/{prefix}/default_example/output"
knn_estimator = trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path)

Deploy the KNN endpoint for retrieving indexes of top K most relevant docuemnts.

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer


def predictor_from_estimator(knn_estimator, instance_type, endpoint_name=None):
    knn_predictor = knn_estimator.deploy(
        initial_instance_count=1, instance_type=instance_type, endpoint_name=endpoint_name
    )
    knn_predictor.serializer = CSVSerializer()
    knn_predictor.deserializer = JSONDeserializer()
    return knn_predictor


instance_type = "ml.m4.xlarge"
endpoint_name = name_from_base(f"jumpstart-example-knn")

knn_predictor = predictor_from_estimator(knn_estimator, instance_type, endpoint_name=endpoint_name)

### 4.4 Retrieve the most relevant documents

Given the embedding of a query, we will query the endpoint to get the indexes of top K most relevant documents and use the indexes to retrieve the corresponded textual documents.

Next, the textual documents are concatenated with maximum length of `MAX_SECTION_LEN`. This is to make sure the context we send into the prompt contains a good enough amount of information all the while not exceeding model's capacity.

In [66]:
KNN_ENDPOINT_NAME = 'jumpstart-example-knn-2023-05-03-08-48-46-033'

In [67]:
from sagemaker import KNNPredictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import numpy as np

knn_predictor = KNNPredictor(KNN_ENDPOINT_NAME, serializer = CSVSerializer(), deserializer = JSONDeserializer())
knn_predictor

In [68]:
MAX_SECTION_LEN = 2000
SEPARATOR = "\n* "


def construct_context(context_predictions_arr, df_knowledge) -> str:
    chosen_sections = []
    chosen_sections_len = 0

    for index in context_predictions_arr:
        # Add contexts until we run out of space.
        document_section = df_knowledge.loc[index]
        chosen_sections_len += len(document_section) + 2
        if chosen_sections_len > MAX_SECTION_LEN:
            break

        chosen_sections.append(SEPARATOR + document_section.replace("\n", " "))
    concatenated_doc = "".join(chosen_sections)
    print(
        f"With maximum sequence length {MAX_SECTION_LEN}, selected top {len(chosen_sections)} document sections: {concatenated_doc}"
    )

    return concatenated_doc, len(chosen_sections)

In [69]:
def construct_context(
    context_predictions_idx, 
    context_prediction_dist, 
    df_knowledge, 
    threshold, 
    context_length
) -> str:
    chosen_sections = []
    chosen_sections_len = 0

    for index, dist in zip(context_predictions_idx, context_prediction_dist):
        # Add contexts until we run out of space.
        document_section = df_knowledge.loc[index]
        chosen_sections_len += len(document_section) + 2
        if dist > threshold:
            break

        chosen_sections.append(SEPARATOR + document_section.replace("\n", " "))
    concatenated_doc = "".join(chosen_sections)[:context_length]
    print(
        f"With maximum sequence length {MAX_SECTION_LEN}, selected top {len(chosen_sections)} document sections: {concatenated_doc}"
    )

    return concatenated_doc, len(chosen_sections)

In [70]:
question

'How can I be sure SageMaker protects my data security and privacy?'

In [51]:
query_response = query_endpoint_with_json_payload(
    question, endpoint_name_embed, content_type="application/x-text"
)
question_embedding = parse_response_multiple_texts(query_response)
np.array(question_embedding)

array([[ 0.01943566, -0.00839817,  0.00582189, ..., -0.00892697,
        -0.00757487, -0.01012382]])

In [52]:
context_predictions_knn = knn_predictor.predict(
    np.array(question_embedding),
    initial_args={"ContentType": "text/csv", "Accept": "application/json; verbose=true"},
)
context_predictions_knn

{'predictions': [{'predicted_label': 44.0,
   'distances': [0.394950270652771,
    0.4050990641117096,
    0.4137035608291626,
    0.4195705056190491,
    0.4368661046028137],
   'labels': [113.0, 145.0, 44.0, 84.0, 130.0]}]}

In [53]:
# Getting the most relevant context using KNN
context_doc_index = context_predictions_knn["predictions"][0]["labels"]
context_doc_dist = context_predictions_knn["predictions"][0]["distances"]
print(context_doc_index, context_doc_dist)

[113.0, 145.0, 44.0, 84.0, 130.0] [0.394950270652771, 0.4050990641117096, 0.4137035608291626, 0.4195705056190491, 0.4368661046028137]


In [54]:
context_retrieve, num_context_doc = construct_context(
    context_doc_index, 
    context_doc_dist, 
    df_knowledge["Answer"],
    threshold=0.4,
    context_length=500,
)

With maximum sequence length 2000, selected top 1 document sections: 
* You can scale down the Amazon SageMaker Asynchronous Inference endpoint instance count to zero in order to save on costs when you are not actively processing requests. You need to define a scaling policy that scales on the "ApproximateBacklogPerInstance" custom metric and set the "MinCapacity" value to zero. For step-by-step instructions, please visit the autoscale an asynchronous endpoint section of the developer guide. 


### 4.5 Combine the retrieved documents, prompt, and question to query the LLM

In [55]:
for model_id in _MODEL_CONFIG_:
    endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]

    prompt = _MODEL_CONFIG_[model_id]["prompt"]

    text_input = prompt.replace("{context}", context_retrieve)
    text_input = text_input.replace("{question}", question)

    payload = {"text_inputs": text_input, **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(f"For model: {model_id}, the generated output is: {generated_texts[0]}\n")

For model: huggingface-text2text-flan-t5-xxl, the generated output is: You can define a scaling policy that scales on the "ApproximateBacklogPerInstance" custom metric and set the "MinCapacity" value to zero.



### 4.6 Clean up

Uncomment below cell to delete the endpoint after testing. 

In [ ]:
# # delete the endpoints created for testing
# for model_id in _MODEL_CONFIG_:
#     endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]
#     sagemaker_session.delete_endpoint(endpoint_name)

# # delete the endpoints hosting the embedding model and knn model
# sagemaker_session.delete_endpoint(endpoint_name_embed)
# knn_predictor.delete_endpoint()